In [104]:
# df.head()

In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


df = pd.read_excel('Cities.xls', index_col=0, skipinitialspace=True) # Read with excel index. 
# Skip all white-spaces.

df = df.set_index('cityID') # Remove the indices and use cityID as index.

# Drop unnecessary columns (that explicitly imply each other)
# cityID implies City
# clusterID implies Typology
# Pollution Index is restricted to be used in prediction.
col2drop = ['City', 'Typology', 'Pollution Index ']
# col2drop = ['City', 'Typology']



df.drop(col2drop, inplace=True, axis=1) # Typology <=> clusterID, City <=> cityID, 
# df.drop("Pollution Index", axis=1)
# df.info()
# df["Pollution Index"] 

In [106]:
def replace_missing_val_rescale(df):
    
    df = df.replace(r'^\s*$', np.NaN, regex=True)
    # print("Initial dataframe shape [rows, columns]:", df.shape)
    n_NaN = df.isna().sum().sum()
    # print(n_NaN) # As you can see the total number of NaNs in the database 3199, so we need to impute.

    # Impute all NaNs over the database.

    df_int = df.select_dtypes(include='integer')
    df_float = df.select_dtypes(include='float')

    df_int = df_int.fillna(df.mode().iloc[0]) # Try using mean/mod for different columns.
    df_float = df_float.fillna(df.mean().iloc[0]) # Try using mean/mod for different columns.

    df[df_int.columns.values.tolist()] = df_int # Use mod for integers,
    df[df_float.columns.values.tolist()] = df_float # Use mean for floats

    # Drop taget variable
    df_targets = df['CO2 Emissions per Capita (metric tonnes)']
    df_inputs = df.drop(columns=['CO2 Emissions per Capita (metric tonnes)'])

    # Normalize data
    from sklearn import preprocessing
    x = df_inputs.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    x_unscaled =  min_max_scaler.inverse_transform(x_scaled)
    df_inputs = pd.DataFrame(x_scaled)

#     df_in_mean = df_inputs.mean()
#     df_in_min = df_inputs.min()
#     df_in_max = df_inputs.max()
    
#     df_in = (df_inputs - df_in_mean)/(df_in_max - df_in_min)
    
    assert df_inputs.isna().sum().sum() == 0, "ERROR!, There still contains NaN values in the df_inputs DataFrame"
    
    return df_inputs, df_targets


In [107]:
# Get DataFrame for only South/ North America
country_list = ["United States", "Canada", "Mexico", "Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador", "Guyana", "Paraguay", "Peru", "Paraguay", "Suriname", "Uruguay", "Venezuela"] 
north_south_america = df[df.Country.isin(country_list)]
# print(north_south_america.head(5))
df2 = df.drop(["Country"], axis=1)
north_south_america = north_south_america.drop(["Country"], axis=1)

In [108]:
df_inputs, df_targets = replace_missing_val_rescale(df2)
# North and South America
df_inputs_nsa, df_targets_nsa = replace_missing_val_rescale(north_south_america)

train_perc = 0.75
tr_slice_ind = int(len(df)*0.75)+1

tr_target = df_targets.head(tr_slice_ind)
tr_inputs = df_inputs.head(tr_slice_ind)

ts_target = df_targets.tail(df_inputs.shape[0]-tr_slice_ind)
ts_inputs = df_inputs.tail(df_inputs.shape[0]-tr_slice_ind)

In [109]:
# # Perform PCA
# from sklearn.decomposition import PCA

# pca = PCA()
# # print(p_df.head())
# # len(pca.components_)
# shape_before = tr_inputs.shape
# pca.fit(tr_inputs)
# expl=pca.explained_variance_ratio_
# cdf=[sum(expl[:i+1]) for i in range(len(expl))]
# plt.plot(range(len(expl)), cdf);
# # print(cdf[:15]) # 15 most explaining components
# pca = PCA(n_components=15) # Looks to about how many components explains 95% of the total variance.
# tr_inputs = pca.fit_transform(tr_inputs) 
# tr_in = pd.DataFrame(tr_inputs)
# p_df = pd.DataFrame(comps)
# shape_after = tr_inputs.shape
# print("input shape before PCA: ", shape_before, "\n", "input shape after PCA:", shape_after)

In [117]:
print(tr_inputs.shape)
tr_inputs.head(50)

(249, 72)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
0,0.545455,0.896624,0.069428,0.005425,0.033333,0.308057,0.215847,0.042347,0.021875,0.062807,0.019676,0.267717,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016863,0.056372,0.000062,0.000000,0.111111,0.222267,0.303344,0.359613,0.405248,0.478483,0.329600,0.045344,0.153327,0.016055,0.127203,0.093657,0.078364,0.065210,0.805151,0.178218,0.675742,0.185185,0.503869,0.315605,0.564083,0.558327,0.865735,0.041698,0.370220,0.822350,0.320829,0.849300,0.133865,0.724427,0.809524,0.078723,0.385022,0.157344,0.109739,0.036735,0.442662,0.155109,0.012693,0.001421,0.007367,0.160848,0.176867,0.360637,0.796264,0.355964,0.081956,0.180085,0.722163,0.425187
1,0.636364,0.843882,0.165652,0.036166,0.051282,0.521327,0.131148,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016863,0.032426,0.006667,0.012845,0.111111,0.453932,0.546621,0.466344,0.388717,0.458172,0.302143,0.090554,0.212685,0.022936,0.139386,0.113821,0.120393,0.107067,0.887750,0.128713,0.451047,0.140212,0.515875,0.285010,0.535844,0.541357,0.800971,1.000000,0.088169,0.914040,0.684456,0.912809,0.126629,0.816285,0.869048,0.091033,0.438123,0.112597,0.185993,0.036148,0.417483,0.157113,0.005946,0.000292,0.002602,0.060387,0.168335,0.363675,0.786174,0.397894,0.082267,0.333173,0.539355,0.558910
2,0.000000,0.058734,0.104750,0.036166,0.769231,0.478673,0.704918,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022740,0.003894,0.243119,0.129379,0.126043,0.152994,0.212399,0.139063,0.356436,0.000865,0.142010,0.000000,0.008135,0.000000,0.000000,0.016720,1.000000,0.300244,0.335244,0.000000,0.003229,0.000000,0.000000,0.018666,0.022496,0.177048,0.101945,0.049892,0.059133,0.832642,0.254547,0.002093,0.000017,0.000000,0.036220,0.010915,0.343161,0.000000,0.273646,0.248398,0.655464,0.275605,0.410312
3,1.000000,0.084388,0.116931,0.036166,0.071384,0.421801,0.653005,0.000000,0.000000,0.000000,0.000000,0.000000,0.070048,1.000000,0.016216,0.060082,0.001410,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.438114,0.537736,0.418373,0.184473,0.067751,0.185780,0.169617,0.219824,0.210222,0.176690,0.296849,0.633663,0.035528,0.175661,0.253908,0.090929,0.252021,0.131704,0.121453,0.022438,0.085435,0.742120,0.614460,0.516685,0.048842,0.540712,0.369048,0.085157,0.058559,0.325636,0.060799,0.005136,0.505583,0.155286,0.019230,0.000165,0.004120,0.078028,0.084511,0.358868,0.157676,0.509057,0.292564,0.624623,0.666710,0.239113
4,1.000000,0.228903,0.661389,0.100687,0.071384,0.545024,0.497268,0.000000,0.000000,0.000000,0.000000,0.000000,0.318841,0.024092,0.287788,0.098452,0.325906,0.152174,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.073197,0.035217,0.142202,0.174942,0.235215,0.239226,0.145008,0.529821,0.980198,0.136873,0.084656,0.000000,0.008135,0.000000,0.000000,0.016720,0.018699,0.088169,0.744986,0.000000,0.516685,0.070550,0.000000,0.642857,0.093713,0.010984,0.598000,0.036618,0.000847,0.344916,0.095738,0.011907,0.000062,0.010799,0.103334,0.447261,0.392551,0.287235,0.801464,0.322960,0.586019,0.791822,0.075617
5,0.545455,0.934599,0.038977,0.009042,0.034615,0.298578,0.251366,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009444,0.050755,0.002222,0.011967,0.222222,0.122982,0.203822,0.235093,0.000000,0.000000,0.000000,0.022201,0.114995,0.011468,0.104885,0.069679,0.060079,0.047583,0.805151,0.178218,0.651551,0.187831,0.473396,0.204589,0.522321,0.532143,0.706371,0.039267,0.433686,0.856734,0.376049,0.836383,0.133865,0.669312,0.809524,0.053346,0.496123,0.148073,0.077845,0.069883,0.633514,0.283571,0

In [111]:
# # Make sure all inputs have the same columns
# ts_in = ts_inputs[list(tr_in.columns)]
# df_inputs_nsa = df_inputs_nsa[list(tr_inputs.columns)]
# print(tr_inputs.shape, ts_inputs.shape, df_inputs_nsa.shape) # All should have same nr. columns

In [114]:
# Perform predictions

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 20)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [3, 5, 8]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]


regr = RandomForestRegressor()

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

regr = RandomizedSearchCV(estimator = regr, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2)

from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(hidden_layer_sizes=(8, 32, 16), 
                   activation='relu',
                   solver = 'adam',
                   alpha = 1e-1,
                   learning_rate='adaptive',
                   learning_rate_init=2.25e-5,
                   max_iter=35000,
                   random_state = 42,
                   shuffle=False).fit(tr_inputs, tr_target)

# pca_test = pca.transform(ts_inputs)
# p_test = pd.DataFrame(pca_test)
est_out = clf.predict(ts_inputs)


In [115]:
# Calculate R2 score.

def adjusted_r2(r_2, y, X):
    adj_r2 = 1 - (1-r_2)*(len(y)-1)/(len(y)-X.shape[1]-1)
    return adj_r2

R_2 = r2_score(ts_target, est_out)
R_2_adj = adjusted_r2(R_2, ts_target, tr_inputs)
print(R_2, R_2_adj)

0.5298220083689269 -3.2316019246796577


In [116]:
# Perform predictions for North and South America
# df_inputs_nsa, df_targets_nsa
est_out = clf.predict(df_inputs_nsa)

R_2 = r2_score(df_targets_nsa, est_out)
R_2_adj = adjusted_r2(R_2, df_targets_nsa, df_inputs_nsa)
print(R_2, R_2_adj)


0.11507195922491054 -2.781056174220837
